In [7]:
import io
import re
import time
import psycopg2
import chardet as cd
import pandas as pd
from sqlalchemy import create_engine

with open("contract_info.csv",'rb') as temp:
    encodingType=cd.detect(temp.read(100000))
    print(encodingType)
'''以正确的解码方式打开文件'''
hw=pd.read_csv("contract_info.csv",encoding=encodingType['encoding'])


{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [8]:
def write_to_table(df, table_name):
    # 链接数据库
    db_engine  = create_engine('postgresql+psycopg2://' + 'postgres' + ':' + 'wyh638' + '@localhost' + ':' + str(5432) + '/' + 'Project1_python')
    string_data_io = io.StringIO()
    
    # 先将DataFrame转化为csv格式
    df.to_csv(string_data_io, sep='|', index=False)
    # 通过pandas的sql_bulider将数据转化为sql中的table的形式
    pd_sql_engine = pd.io.sql.pandasSQL_builder(db_engine)
    table = pd.io.sql.SQLTable(table_name, pd_sql_engine, frame=df,
                               index=False, if_exists='append',schema = 'public')
    table.create()
    string_data_io.seek(0)
    
    # 通过sql的COPY命令将table复制到数据库中
    with db_engine.connect() as connection:
        with connection.connection.cursor() as cursor:
            copy_cmd = "COPY public.%s FROM STDIN HEADER DELIMITER '|' CSV" %table_name
            cursor.copy_expert(copy_cmd, string_data_io)
        connection.connection.commit()

In [4]:
time_start=time.time()
supply_center=hw[['supply center','director']].copy()
supply_center.drop_duplicates(subset='supply center',inplace=True)
write_to_table(supply_center,'supply_center')
time_end=time.time()
print('shape of data: ', supply_center.shape[0], ' rows')
print('totally cost',time_end-time_start, 's')

shape of data:  7  rows
totally cost 0.09303402900695801 s


In [40]:
time_start=time.time()
salesman=hw[['salesman number','salesman','gender','age','mobile phone']].copy()
salesman.drop_duplicates(subset='salesman number',inplace=True)
salesman.drop_duplicates(subset='mobile phone',inplace=True)
#write_to_table(salesman,'salesman')
salesman.to_csv('salesman.csv',index=False)
time_end=time.time()
print('shape of data: ', salesman.shape[0], ' rows')
print('totally cost',time_end-time_start, 's')

shape of data:  990  rows
totally cost 0.06799864768981934 s


In [41]:
time_start=time.time()
product=hw[['product model','product code','product name','unit price']].copy()
product.drop_duplicates(subset='product model',inplace=True)
write_to_table(product,'product')
time_end=time.time()
print('shape of data: ', product.shape[0], ' rows')
print('totally cost',time_end-time_start, 's')

shape of data:  961  rows
totally cost 0.050043344497680664 s


In [8]:
time_start=time.time()
enterprise=hw[['client enterprise','country','city','industry']].copy()
enterprise.drop_duplicates(subset='client enterprise',inplace=True)
write_to_table(enterprise,'enterprise')
time_end=time.time()
print('shape of data: ',enterprise.shape[0], ' rows')
print('totally cost',time_end-time_start, 's')



shape of data:  167  rows
totally cost 0.08951878547668457 s


In [3]:
time_start=time.time()
contract=hw[['contract number','client enterprise','supply center','contract date']].copy()
contract.drop_duplicates(subset='contract number',inplace=True)
#write_to_table(contract,'contract')
time_end=time.time()
contract.to_csv('contract.csv',index=False)
print('shape of data: ',contract.shape[0], ' rows')
print('totally cost',time_end-time_start, 's')

shape of data:  5000  rows
totally cost 0.010005474090576172 s


In [12]:
import numpy as np
time_start=time.time()
orders_f=hw[['contract number','product model','quantity','estimated delivery date','lodgement date','salesman number']].copy()
orders_f['lodgement date']=pd.to_datetime(hw['lodgement date'],format='%Y/%m/%d')
orders_f.loc[orders_f['lodgement date']>pd.to_datetime('2022-03-02'),['lodgement date']]=None
data=np.arange(1,len(orders_f)+1).reshape(len(orders_f),-1)
data
data_f=pd.DataFrame(data)

orders=pd.concat([data_f,orders_f],axis=1)
orders.to_csv('orders.csv',index=False)
write_to_table(orders,'orders')
time_end=time.time()
print('shape of data: ',orders.shape[0], ' rows')
print('totally cost',time_end-time_start, 's')

shape of data:  50000  rows
totally cost 2.936455726623535 s
